# Example usage

You can try `xarray_simlab_ode` by running the model presented in the Readme file.

In [ ]:
import xso

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
@xso.component
class Variable:
    value = xso.variable(description='basic state variable', attrs={'units':'µM'})

In [ ]:
@xso.component
class LinearGrowth:
    var_ext = xso.variable(foreign=True, flux='growth', description='external state variable')
    rate = xso.parameter(description='linear growth rate', attrs={'units':'$d^{-1}$'})

    @xso.flux
    def growth(self, var_ext, rate):
        return var_ext * rate

In [ ]:
model = xso.create({'Var':Variable,'Growth':LinearGrowth}, time_unit='d')

In [ ]:
print(model)

In [ ]:
input_ds = xso.setup(solver='solve_ivp',
                     model=model,
                     time=np.arange(1,10,.1),
                     input_vars={
                         'Var':{'value_label':'X', 'value_init':1},
                         'Growth':{'var_ext':'X', 'rate':1.},
                     })

In [ ]:
with model:
    output_ds = input_ds.xsimlab.run()

In [ ]:
output_ds

In [ ]:
output_ds.Var__value.plot()

In [ ]:
with model:
    batchout_ds = (input_ds
     .xsimlab.update_vars(
         input_vars={'Growth': {'rate': ('batch', [0.9, 1.0, 1.1, 1.2])}}
     )
     .xsimlab.run(parallel=True, batch_dim='batch')
     .swap_dims({'batch':'Growth__rate'})
     .Var__value.plot.line(x='time')
     )